<h1><center>2021 BRFSS Data: Part 3</center></h1>


#### About the data:
- The dataset used in this project is the 2021 BRFSS Data.
- The Behavioral Risk Factor Surveillance System (BRFSS) is a collaborative project between all the states in the United States and participating US territories and the Centers for Disease Control and Prevention (CDC).
- It is used to collect prevalence data among adult U.S. residents regarding their risk behaviors and preventive health practices that can affect their health status. Respondent data are forwarded to CDC to be aggregated for each state, returned with standard tabulations, and published at year's end by each state. 
- To get the database used, __[Click Here](https://www.cdc.gov/brfss/annual_data/annual_2021.html)__
- For the codebook of the database, __[Click Here](https://www.cdc.gov/brfss/annual_data/2021/pdf/codebook21_llcp-v2-508.pdf)__


#### The project is:
- __Part 1. Preparing:__ In this part, I will choose what columns will be left to analyze, change the values in them to more human-readable form according to the BRFSS codebook.<br>
- __Part 2. EDA:__ In this part, I will do some exploratory data analysis, trying to find insights from the data.<br>
- __Part 3. Statistics:__ In this part, I will do some statistical hypotheses testing to evaluate some of the insights found in part 2, I will also conduct regression analysis.

In [1]:
# Import the packages that will be needed in this analysis
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Define some setting
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = (10,8)
plt.rcParams['axes.titlesize'] = 18

In [2]:
# Import the datafame
df = pd.read_pickle('LLCP2021_Preprocessed.pkl')

# Define columns' lists
diseases = ['HTN', 'DM', 'Asthma', 'COPD', 'Arthritis', 'Kidney_Disease', 'Depression', 'MI', 'CHD', 'Stroke']
cat_vars = [col for col in df.columns if (df[col].dtypes == 'object' or df[col].dtypes == 'category') and col not in diseases]
num_vars = [col for col in df.columns if col not in cat_vars + diseases]
cat_cols = diseases + cat_vars

df.sample(5)

,Urban_Rural,Gender,Age_Category,Education,Martial_Status,Income_Category,Veteran,Height,Weight,BMI,BMI_Category,Smoking,Alcohol_Drinks,Insurance,HTN,MI,CHD,Stroke,DM,DM_Age,BGM_Weekly,A1C,Insulin,Retinopathy_Counseled,Feet_Check,Asthma,COPD,Arthritis,Kidney_Disease,Depression
274513,Urban counties,Female,35 to 44,Attended College or Technical School,Never married,"$15,000 to < $25,000",No,165.0,NaN,NaN,NaN,Never smoked,NaN,NaN,Yes,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,No,No,No
373570,Urban counties,Female,35 to 44,Graduated from College or Technical School,Married,"$100,000 to < $200,000",Yes,155.0,87.540,36.47,Obese,Never smoked,1.0,Yes,Yes,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,No,No,Yes,No,Yes
65938,Urban counties,Male,25 to 34,Attended College or Technical School,Married,"$15,000 to < $25,000",No,183.0,90.720,27.12,Overweight,Former smoker,2.0,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,No,No
21189,Urban counties,Female,45 to 54,Attended College or Technical School,Married,"$50,000 to < $100,000",No,163.0,72.570,27.46,Overweight,Current smoker,2.0,Yes,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,Yes,No,No
372417,Urban counties,Male,35 to 44,Attended College or Technical School,Divorced,NaN,No,183.0,136.065,43.36,Obese,Former smoker,4.0,Yes,Yes,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,No,No


In [3]:
# Check if there is any outliers in the database.
# And, if they do exist, replace them with the threshold (either upper or lower limits).

def outliers(dataframe, column):
    
    """
    Purpose:
    The function outliers takes a pandas dataframe and a column as input, and returns the dataframe with the outliers in the specified column replaced by the upper and lower limits calculated using the interquartile range (IQR).

    Inputs:
    dataframe: a pandas dataframe
    column: the name of a column in the dataframe. The column should contain numerical values.

    Outputs:
    The function returns the dataframe with outliers in the specified column replaced by the upper and lower limits, mentioning numbers of outliers that were replaced in the column.
    If the column does not have any outliers, the function returns the original dataframe with a message indicating that no changes were made.

    Methodology:
    The first quartile (q1) and the third quartile (q3) are calculated for the specified column.
    The interquartile range (iqr) is calculated as the difference between q3 and q1.
    The upper limit is calculated as q3 + (1.5 * iqr), and the lower limit is calculated as q1 - (1.5 * iqr).
    If the dataframe contains any values outside the upper or lower limits, the outliers in the specified column are replaced by the upper and lower limits.
    The dataframe is returned, with a message indicating that the outliers were replaced if applicable.  
    """
    
    q1 = dataframe[column].quantile(0.25)
    q3 = dataframe[column].quantile(0.75)
    iqr = q3 - q1
    lower_limit = q1 - (1.5 * iqr)
    upper_limit = q3 + (1.5 * iqr)
    outliers_replaced = len(dataframe[(dataframe[column] < lower_limit) | (dataframe[column] > upper_limit)])
    if dataframe[(dataframe[column] > upper_limit) | (dataframe[column] < lower_limit)].any(axis=None):
        dataframe.loc[(dataframe[column] < lower_limit), column] = lower_limit
        dataframe.loc[(dataframe[column] > upper_limit), column] = upper_limit
        print(f"- {column} column had {outliers_replaced} outliers; these outliers were replaced with upper/lower limits.")
    else:
        print(f"- {column} column did not have any outliers; no changes were made.")
    
for col in num_vars:
    outliers(df, col)
    
df.describe().T.round(2)

- Height column did not have any outliers; no changes were made.
- Weight column did not have any outliers; no changes were made.
- BMI column did not have any outliers; no changes were made.
- Alcohol_Drinks column did not have any outliers; no changes were made.
- DM_Age column did not have any outliers; no changes were made.
- BGM_Weekly column did not have any outliers; no changes were made.
- A1C column did not have any outliers; no changes were made.
- Feet_Check column did not have any outliers; no changes were made.


,count,mean,std,min,25%,50%,75%,max
Height,401584.0,170.29,10.63,140.50,163.00,170.00,178.00,200.50
Weight,386935.0,83.01,20.26,27.23,68.04,81.65,95.25,136.06
BMI,380722.0,28.44,5.97,12.72,24.21,27.44,31.87,43.36
Alcohol_Drinks,199971.0,2.08,1.39,0.00,1.00,2.00,3.00,6.00
DM_Age,52593.0,49.70,14.55,10.00,40.00,50.00,60.00,90.00
BGM_Weekly,18487.0,9.78,8.93,0.02,2.00,7.00,14.00,32.00
A1C,20557.0,2.52,1.63,0.00,1.00,2.00,4.00,8.50
Feet_Check,21031.0,1.85,1.70,0.00,1.00,1.00,3.00,6.00


# Hypothesis Testing:

### Random hypotheses testings to spot any statistically significant variables that affects diseases: 

- For numerical columns, the test checks the difference of mean between having being diagnosed with each disease and not being diagnosed.
- For categorical columns, if the values in the categorical column are two, then the test checks the difference of proportions between each variable in patients diagnosed with each disease.
- For both of them, the tests check if the difference are statistically significant, then the results are added in a dataframe called 'p_diseases'.

### Null and Alternative Hypotheses:
- __Null Hypothesis $H_0$:__ There is no difference in the proportions of variables between those who are diagnosed and those who are not.
- __Alternative Hypothesis $H_1$:__ Proportions of variables between those who are diagnosed and those who are not is different.

In [4]:
df_ = df.stack().str.replace(' ', '_').unstack().copy()

p_diseases = pd.DataFrame(columns=['Disease', 'Variable','P_Value','Higher_Effect','Difference','Statistical_Significance','Observations'])

for disease in diseases:

    for col in cat_vars:
        temp = df_.dropna(subset=[disease]).copy()
        values = temp[col].dropna().unique()
        
        if len(values) == 2:
            #Number of patients with each specific variable who have the disease
            n_v0 = temp[(temp[disease] == 'Yes') & (temp[col] == values[0])].shape[0]
            n_v1 = temp[(temp[disease] == 'Yes') & (temp[col] == values[1])].shape[0]
            
            #Total number of people with a specific variable
            total_v0 = temp[temp[col] == values[0]].shape[0] 
            total_v1 = temp[temp[col] == values[1]].shape[0]
            pvalue = sm.stats.proportions_ztest([n_v0, n_v1], [total_v0, total_v1], alternative='two-sided')[1]
            
            if pvalue <= 0.05:
                if (n_v0/total_v0) > (n_v1/total_v1):
                    higher = str(values[0])
                else:
                    higher = str(values[1])
                diff = (max((n_v0/total_v0), (n_v1/total_v1)) - min((n_v0/total_v0), (n_v1/total_v1))) * 100
                p_diseases.loc[len(p_diseases.index)] = [disease, col, pvalue, higher, round(diff, 2), 'Yes', len(temp)]
            else:
                p_diseases.loc[len(p_diseases.index)] = [disease, col, pvalue, np.nan, np.nan, 'No', len(temp)]

    for col in num_vars:
        temp = df[[disease, col]].dropna(subset=[disease]).copy()
        temp[col] = temp[col].fillna(temp[col].mean())
        if len(temp[disease].unique()) == 2:
            pvalue = sm.stats.ztest(temp[temp[disease] == 'No'][col], temp[temp[disease] == 'Yes'][col], alternative = 'two-sided')[1]

            if pvalue <= 0.05:
                if temp[temp[disease] == 'No'][col].mean() > temp[temp[disease] == 'Yes'][col].mean():
                    higher = 'No'
                else:
                    higher = 'Yes'
                diff = max(temp[temp[disease] == 'No'][col].mean(), temp[temp[disease] == 'Yes'][col].mean()) - min(temp[temp[disease] == 'No'][col].mean(), temp[temp[disease] == 'Yes'][col].mean()) 
                p_diseases.loc[len(p_diseases.index)] = [str(disease), str(col), pvalue, higher, round(diff, 2), 'Yes', len(temp)]
            else:
                p_diseases.loc[len(p_diseases.index)] = [str(disease), str(col), pvalue, np.nan, np.nan, 'No', len(temp)]

p_diseases = p_diseases.sort_values(by=['Statistical_Significance', 'Disease']).reset_index(drop=True)

print(f"Total number of {len(p_diseases)} hypothesis testings were conducted.\n\
{len(p_diseases[p_diseases['Statistical_Significance'] == 'Yes'])} of them proved statistical significance, \
and {len(p_diseases[p_diseases['Statistical_Significance'] == 'No'])} showed no statistical significance.")

p_diseases

Total number of 140 hypothesis testings were conducted.
131 of them proved statistical significance, and 9 showed no statistical significance.


,Disease,Variable,P_Value,Higher_Effect,Difference,Statistical_Significance,Observations
0,Arthritis,Retinopathy_Counseled,1.546039e-01,NaN,NaN,No,416249
1,Asthma,Alcohol_Drinks,9.006824e-01,NaN,NaN,No,417394
2,DM,Insulin,NaN,NaN,NaN,No,418155
3,DM,Retinopathy_Counseled,NaN,NaN,NaN,No,418155
4,DM,DM_Age,1.000000e+00,NaN,NaN,No,418155
5,DM,BGM_Weekly,1.000000e+00,NaN,NaN,No,418155
6,DM,A1C,1.000000e+00,NaN,NaN,No,418155
7,DM,Feet_Check,1.000000e+00,NaN,NaN,No,418155
8,Kidney_Disease,Urban_Rural,1.346233e-01,NaN,NaN,No,417329
9,Arthritis,Urban_Rural,1.050943e-142,Rural_counties,5.29,Yes,416249


- __The main findings from the conducted tests above are:__
    - Results that had statistically significant difference:
        - Diabetic patients who also suffer from cardiovascular complications or kidney diseases have higher probability of checking for retinopathy than those who are not.
        - Diabetic patients who also suffer from cardiovascular complications or kidney diseases have higher probability of taking insulin than those who are not.
        - Veterans are less likely to be diagnosed depression or Asthma, but are more likely to be diagnosed with Arthritis, COPD, Diabetes, Hypertension and cardiovascular complications.
        - Proportion of people living in rural areas who suffer from Asthma or Hypertension is around 5% higher than those who live in urban areas.
        - Proportion of females who are diagnosed with Arthritis, Asthma, & Depression is more than males.
        - Proportion of males who are diagnosed with cardiovascular diseases (HTN, MI, CHD) is more than females.
        - In all diseases in the datasets, patients who were diagnosed had a higher average BMI than those who were not diagnosed.
        - People who are any kind of insurance were more likely to be diagnosed in all the diseases in the dataset. Most probable reason is that those who do not have insurance suffer from those disease but are undiagnosed, which would put them at risk for complications, but more data is needed to confirm this hypothesis. 
        <br><br>
    - Results that did not have statistically significant difference:
        - There is no difference in the average alcohol drinks between those diagnosed with Asthma and those who are not.
        - There is no difference in the proportion of patients having kidney disease between those who live in rural counties and urban counties.

### Random hypotheses testings to spot any statistically significant difference between the numerical and categorical variables: 

- The next tests take each of the numerical variables in the dataset and test it against all the categorical variables (except the diseases columns) and check if there is any difference of the mean of that numerical variable in the two groups of the categorical variable.  
- For both of them, the tests check if the difference are statistically significant, then the results are added in a dataframe called 'p_numerics'.

### Null and Alternative Hypotheses:
- __Null Hypothesis $H_0$:__ There is no difference in the average of the numerical variable in the two groups of the categorical variables.
- __Alternative Hypothesis $H_1$:__ The average of the numerical variable in the two groups of the categorical variables is different.

In [5]:
p_numerics = pd.DataFrame(columns=['Categorical_Variable', 'Numeric_Variable', 'Higher_Effect', 'Difference', 'P_Value','Statistical_Significance'])

for cat in cat_vars:
    for num in num_vars:
        values = df[cat].dropna().unique()
        if len(values) == 2:
            pvalue = sm.stats.ztest(df[df[cat] == values[0]][num].dropna(), df[df[cat] == values[1]][num].dropna(), alternative = 'two-sided')[1]
            if pvalue <= 0.05:
                if df[df[cat] == values[0]][num].dropna().mean() > df[df[cat] == values[1]][num].dropna().mean():
                    higher = str(values[0])
                else:
                    higher = str(values[1])
                diff = max(df[df[cat] == values[0]][num].dropna().mean(), df[df[cat] == values[1]][num].dropna().mean()) - min(df[df[cat] == values[0]][num].dropna().mean(), df[df[cat] == values[1]][num].dropna().mean()) 
                p_numerics.loc[len(p_numerics.index)] = [str(cat), str(num), higher, round(diff, 2), pvalue, 'Yes']
            else:
                p_numerics.loc[len(p_numerics.index)] = [str(cat), str(num), np.nan, np.nan, pvalue, 'No']

p_numerics = p_numerics.sort_values(by=['Statistical_Significance', 'Categorical_Variable'], ascending=[False, True]).reset_index(drop=True)

print(f"Total number of {len(p_numerics)} hypothesis testings were conducted.\n\
{len(p_numerics[p_numerics['Statistical_Significance'] == 'Yes'])} of them proved statistical significance, \
and {len(p_numerics[p_numerics['Statistical_Significance'] == 'No'])} showed no statistical significance.")

p_numerics

Total number of 48 hypothesis testings were conducted.
38 of them proved statistical significance, and 10 showed no statistical significance.


,Categorical_Variable,Numeric_Variable,Higher_Effect,Difference,P_Value,Statistical_Significance
0,Gender,Height,Male,14.78,0.000000e+00,Yes
1,Gender,Weight,Male,14.66,0.000000e+00,Yes
2,Gender,BMI,Male,0.23,3.946987e-32,Yes
3,Gender,Alcohol_Drinks,Male,0.57,0.000000e+00,Yes
4,Gender,DM_Age,Male,0.83,5.882119e-11,Yes
5,Gender,BGM_Weekly,Female,0.68,2.037451e-07,Yes
6,Gender,A1C,Female,0.10,4.451884e-06,Yes
7,Insulin,Height,Yes,0.47,3.342526e-03,Yes
8,Insulin,Weight,Yes,2.62,1.803950e-16,Yes
9,Insulin,BMI,Yes,0.71,6.071964e-14,Yes


- __The main findings from the conducted tests above are:__
    - Results that had statistically significant difference:
        - For diabetic patients, females tend to get diagnosed with Diabetes at lower age compared to men, and their average blood glucose and A1C measurement is higher.
        - Diabetic patients who take insulin for their diabetes usually get diagnosed at lower age than those who do not with an average of 8 years. This is most probably because those who are diagnosed at lower age are type-1 diabetes, but unfortunately, more data is required to prove that hypothesis.
        - Diabetic patients who take insulin on average have higher weight and BMI than those who are not.
        - On average, Diabetic patients who have insurance get diagnosed at age 8 years younger than those who do not have insurance.
        - Veterans tend to be diagnosed with Diabetes at considerably higher age, which could be related to better physical health of veterans.
        <br><br>
    - Results that did not have statistically significant difference:
        - For diabetic patients, average alcoholic drinks are the same between those who take Insulin and those who do not.
        - For diabetic patients, average blood glucose measurement is the same between those who have insurance and those who do not.
        - For diabetic patients, average blood glucose measurement, A1C measurement, feet checking, is the same between those who live in urban areas or rural areas.

### Difference in age of diabetes diagnosis between males and females:

- __Null Hypothesis $H_0$:__ Age of diabetes' diagnosis does not differ between males and females.
- __Alternative Hypothesis $H_1$:__ Females tend to get diagnosed with DM at slightly lower age compared to males.

In [6]:
pvalue = sm.stats.ztest(df[df['Gender'] == 'Female']['DM_Age'].dropna(),
                        df[df['Gender'] == 'Male']['DM_Age'].dropna(),
                        alternative = 'smaller')[1]

if pvalue <= 0.05:
    print(f"The p-value of the conducted hypothesis testing is {pvalue}.\nThis means that the alternative hypothesis is accepted and there is evidence that females do tend to get diagnosed with DM at slightly lower age compared to males.")
else:
    print(f"The p-value of the conducted hypothesis testing is {pvalue}.\nThis means that we fail to reject the null hypothese and there is no statistical evidence that females get diagnosed with DM at slightly lower age compared to males.") 

The p-value of the conducted hypothesis testing is 2.9410596099104626e-11.
This means that the alternative hypothesis is accepted and there is evidence that females do tend to get diagnosed with DM at slightly lower age compared to males.



# Insights found in this project:<br>

- __DM:__
    - Diabetic patients who also suffer from cardiovascular complications or kidney diseases have higher probability of checking for retinopathy than those who are not.
    - Diabetic patients who also suffer from cardiovascular complications or kidney diseases have higher probability of taking insulin than those who are not.
    - Diabetic patients who take insulin for their diabetes usually get diagnosed at lower age than those who do not with an average of 8 years. This is most probably because those who are diagnosed at lower age are type-1 diabetes, but unfortunately, more data is required to prove that hypothesis.
    - Diabetic patients who take insulin on average have higher weight and BMI than those who are not.
    - On average, Diabetic patients who have insurance get diagnosed at age 8 years younger than those who do not have insurance.
    - For diabetic patients, average alcoholic drinks are the same between those who take Insulin and those who do not.
    - For diabetic patients, average blood glucose measurement is the same between those who have insurance and those who do not.
- __Veterans__:
    - Veterans are less likely to be diagnosed depression or Asthma, but are more likely to be diagnosed with Arthritis, COPD, Diabetes, Hypertension and cardiovascular complications.
    - Veterans tend to be diagnosed with Diabetes at considerably higher age, which could be related to better physical health of veterans.
<br><br>
- __Rural areas:__
    - Proportion of people living in rural areas who suffer from Asthma or Hypertension is around 5% higher than those who live in urban areas.
    - There is no difference in the proportion of patients having kidney disease between those who live in rural counties and urban counties.
    - For diabetic patients, average blood glucose measurement, A1C measurement, feet checking, is the same between those who live in urban areas or rural areas.
<br><br>
- __Gender:__
    - Proportion of females who are diagnosed with Arthritis, Asthma, & Depression is more than males.
    - Proportion of males who are diagnosed with cardiovascular diseases (HTN, MI, CHD) is more than females.
    - For diabetic patients, females tend to get diagnosed with Diabetes at lower age compared to men, and their average blood glucose and A1C measurement is higher.
<br><br>
- __Other Insights:__
    - In all diseases in the datasets, patients who were diagnosed had a higher average BMI than those who were not diagnosed.
    - There is no difference in the average alcohol drinks between those diagnosed with Asthma and those who are not.
    - People who are any kind of insurance were more likely to be diagnosed in all the diseases in the dataset. Most probable reason is that those who do not have insurance suffer from those disease but are undiagnosed, which would put them at risk for complications, but more data is needed to confirm this hypothesis.